In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-18 05:35:47--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-05-18 05:35:49 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [0]:
#replace <your ...> with your database connection information
jdbc_url = "jdbc:postgresql://<your connection string>:5432/<your database-name>"
config = {"user":"<your database user>",
          "password": "<your database password>",
          "driver":"org.postgresql.Driver"}

In [5]:
video_game_reviews = spark.read.jdbc(url=jdbc_url,table="video_game_vine_reviews", properties=config)
video_game_reviews.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2XVQGWM2ZBPVC|          2|            0|          1|   Y|
| RY4HCU6JP68XV|          4|            3|          4|   Y|
| RP4G580BS1BUW|          3|            3|          5|   Y|
| R7UVLCPUNNEH5|          5|            0|          0|   Y|
|R2KC8PGKFPTDZE|          5|            0|          0|   Y|
| R86E4NB1M93UD|          5|            0|          0|   Y|
|R3DG8ASP4P2BYH|          3|            0|          0|   Y|
|R23DJKDOCX97I0|          1|            0|          0|   Y|
|R2PJOCRWZX4RQZ|          2|            1|          2|   Y|
|R2RRO66EPNH0SB|          5|            3|         15|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [9]:
software_reviews = spark.read.jdbc(url=jdbc_url, table="software_vine_reviews", properties=config)
software_reviews.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1P0EO605W4X4E|          4|            1|          2|   Y|
|R2C6QMNTMOEWO1|          5|            2|         15|   Y|
| RKS9UBOIVBWAS|          4|            1|          2|   Y|
| R3P0YB3905OX1|          5|            2|          2|   Y|
|R1PT5SI2DVKACS|          5|            1|          1|   Y|
|R1H43S0F8CD17E|          1|            5|          5|   Y|
|R2L43434P0K0VT|          5|            0|          0|   Y|
|R35ZJHQME3IR49|          5|           49|         51|   Y|
|R1LJO7L4Y1N452|          5|            5|          8|   Y|
|R1XI4RP8200IMK|          1|           44|         48|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



## Vine reviews analysis

In [10]:
video_game_reviews.describe().show()

+-------+--------------+------------------+------------------+------------------+----+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|vine|
+-------+--------------+------------------+------------------+------------------+----+
|  count|          4290|              4290|              4290|              4290|4290|
|   mean|          null|4.0748251748251745| 2.348717948717949|3.2783216783216784|null|
| stddev|          null|0.9183222304948375|12.580450532105276|13.769965761758117|null|
|    min|R10079FVQQF194|                 1|                 0|                 0|   Y|
|    max| RZXZZ98IW0F2G|                 5|               347|               362|   Y|
+-------+--------------+------------------+------------------+------------------+----+



In [11]:
software_reviews.describe().show()

+-------+--------------+------------------+------------------+-----------------+-----+
|summary|     review_id|       star_rating|     helpful_votes|      total_votes| vine|
+-------+--------------+------------------+------------------+-----------------+-----+
|  count|         10415|             10415|             10415|            10415|10415|
|   mean|          null| 3.839174267882861|3.1916466634661544|3.871435429668747| null|
| stddev|          null|1.1056021641842035| 24.91389719042119| 25.6265464330581| null|
|    min|R1000QJKXGZITK|                 1|                 0|                0|    Y|
|    max| RZZ8LCK0XZBRA|                 5|              1231|             1247|    Y|
+-------+--------------+------------------+------------------+-----------------+-----+



In [36]:
from pyspark.sql.functions import round
video_game_reviews_count = video_game_reviews.count()
video_game_helpful_vote = video_game_reviews.groupBy("helpful_votes").count()
video_game_helpful_vote = video_game_helpful_vote.orderBy(video_game_helpful_vote["count"].desc())
video_game_helpful_vote.withColumn('percentage', round(video_game_helpful_vote["count"]/video_game_reviews_count,4)).show()

+-------------+-----+----------+
|helpful_votes|count|percentage|
+-------------+-----+----------+
|            0| 2314|    0.5394|
|            1|  905|     0.211|
|            2|  370|    0.0862|
|            3|  201|    0.0469|
|            4|  136|    0.0317|
|            5|   69|    0.0161|
|            6|   51|    0.0119|
|            7|   40|    0.0093|
|            8|   21|    0.0049|
|           10|   18|    0.0042|
|            9|   16|    0.0037|
|           12|   14|    0.0033|
|           11|   11|    0.0026|
|           13|   10|    0.0023|
|           16|    9|    0.0021|
|           19|    7|    0.0016|
|           14|    6|    0.0014|
|           21|    6|    0.0014|
|           15|    5|    0.0012|
|           23|    5|    0.0012|
+-------------+-----+----------+
only showing top 20 rows



In [37]:
software_reviews_count = software_reviews.count()
software_helpful_vote = software_reviews.groupBy("helpful_votes").count()
software_helpful_vote = software_helpful_vote.orderBy(software_helpful_vote["count"].desc())
software_helpful_vote.withColumn('percentage', round(software_helpful_vote["count"]/software_reviews_count,4)).show()

+-------------+-----+----------+
|helpful_votes|count|percentage|
+-------------+-----+----------+
|            0| 5054|    0.4853|
|            1| 2315|    0.2223|
|            2| 1069|    0.1026|
|            3|  572|    0.0549|
|            4|  347|    0.0333|
|            5|  223|    0.0214|
|            6|  147|    0.0141|
|            7|  111|    0.0107|
|            8|   65|    0.0062|
|            9|   56|    0.0054|
|           11|   50|    0.0048|
|           10|   41|    0.0039|
|           13|   24|    0.0023|
|           14|   24|    0.0023|
|           12|   24|    0.0023|
|           15|   18|    0.0017|
|           19|   16|    0.0015|
|           16|   15|    0.0014|
|           17|   13|    0.0012|
|           23|   12|    0.0012|
+-------------+-----+----------+
only showing top 20 rows



In [38]:
video_game_star_rating = video_game_reviews.groupBy("star_rating").count()
video_game_star_rating = video_game_star_rating.orderBy(video_game_star_rating["count"].desc())
video_game_star_rating.withColumn('percentage', round(video_game_star_rating["count"]/video_game_reviews_count,4)).show()

+-----------+-----+----------+
|star_rating|count|percentage|
+-----------+-----+----------+
|          4| 1711|    0.3988|
|          5| 1607|    0.3746|
|          3|  718|    0.1674|
|          2|  194|    0.0452|
|          1|   60|     0.014|
+-----------+-----+----------+



In [40]:
software_star_rating = software_reviews.groupBy("star_rating").count()
software_star_rating = software_star_rating.orderBy(software_star_rating["count"].desc())
software_star_rating.withColumn('percentage', round(software_star_rating["count"]/software_reviews_count,4)).show()

+-----------+-----+----------+
|star_rating|count|percentage|
+-----------+-----+----------+
|          4| 3755|    0.3605|
|          5| 3381|    0.3246|
|          3| 2006|    0.1926|
|          2|  769|    0.0738|
|          1|  504|    0.0484|
+-----------+-----+----------+

